In [17]:
import torch
import numpy as np
import pandas as pd

In [18]:
test = torch.from_numpy(pd.read_csv("test.csv").values).float()
train = torch.from_numpy(pd.read_csv("train.csv").values).float()

testX = test[:,:-1]
trainX = train[:,:-1]
testY = test[:,-1]
trainY = train[:,-1]

In [19]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=torch.float)

print("Predicted deaths for first census tract:",(testX[0] @ coef).item())
print("Average predicted deaths over whole set", torch.mean(testX @ coef).item())

Predicted deaths for first census tract: 9.844000816345215
Average predicted deaths over whole set 12.07363224029541


In [20]:
x = torch.tensor(0.0).requires_grad_(True)
# class Y(torch.nn.Module):
#     def __init__(self, x):
#         super(Y, self).__init__()
#         self.x_ = x.requires_grad_(True)
#     def forward(self):
#         return self.x_*self.x_ - 8*self.x_ + 19
def y_fn(x_):
    return x_*x_ - 8*x_ + 19
print("Q6: y=" ,y_fn(x).item())

import torch.optim as optim
optimizer = optim.SGD([x],lr=0.1,maximize=False)
epochs = 100
for t in range(epochs):
    optimizer.zero_grad()
    loss = y_fn(x)
    loss.backward()
    if (t+1) % 10 == 0:
        print("Epoch", str(t+1) , "Loss:", loss.item())
    optimizer.step()

print("Q7: Value of x to minimize y:",x.item())

Q6: y= 19.0
Epoch 10 Loss: 3.2882308959960938
Epoch 20 Loss: 3.003323554992676
Epoch 30 Loss: 3.0000381469726562
Epoch 40 Loss: 3.0
Epoch 50 Loss: 3.0
Epoch 60 Loss: 3.0
Epoch 70 Loss: 3.0
Epoch 80 Loss: 3.0
Epoch 90 Loss: 3.0
Epoch 100 Loss: 3.0
Q7: Value of x to minimize y: 3.999999523162842


In [21]:
dim = trainX.shape[1]
coef = torch.zeros((dim, 1)).requires_grad_(True)

torch.random.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

op = optim.SGD([coef], lr=0.000002, maximize=False)
loss_fn = torch.nn.MSELoss()

for t in range(500):
    for x, y in dl:
        op.zero_grad()
        loss = loss_fn((x @ coef).flatten(), y)
        loss.backward()
        # print(loss.item())
        op.step()
    if (t+1) % 50 == 0:
        print("Epoch", str(t+1) , "Loss:", loss.item())


Epoch 50 Loss: 17.75841522216797
Epoch 100 Loss: 31.456315994262695
Epoch 150 Loss: 27.300701141357422
Epoch 200 Loss: 30.77610969543457
Epoch 250 Loss: 46.19178771972656
Epoch 300 Loss: 23.183555603027344
Epoch 350 Loss: 27.778663635253906
Epoch 400 Loss: 25.559171676635742
Epoch 450 Loss: 20.10915756225586
Epoch 500 Loss: 15.432137489318848


In [22]:
print("Total loss over train:", loss_fn((trainX @ coef).flatten(), trainY).item())
print("Total loss over test:", loss_fn((testX @ coef).flatten(), testY).item())

Total loss over train: 26.8113956451416
Total loss over test: 29.05854606628418
